<div class="list-group" id="list-tab" role="tablist">
<h2 id="intermediate-image-exploration" class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Intermediate Image Exploration</center></h2>

### Image Dimension

We shall check if the provided images have the same dimension or not. I'll use Dask to parallelize the operation and speed things up.

In [ ]:
# get image dimensions
def get_dims(file):
    img = cv2.imread(file)
    h,w = img.shape[:2]
    return h,w

# parallelize using Dask.bag
filelist = train_images_path
dimsbag = bag.from_sequence(filelist).map(get_dims)
with diagnostics.ProgressBar():
    dims = dimsbag.compute()
    
dim_df = pd.DataFrame(dims, columns=['height', 'width'])
sizes = dim_df.groupby(['height', 'width']).size().reset_index().rename(columns={0:'count'})
sizes.hvplot.scatter(x='height', y='width', size='count', xlim=(0,1200), ylim=(0,1200), grid=True, xticks=2, 
        yticks=2, height=500, width=600).options(scaling_factor=0.1, line_alpha=1, fill_alpha=0)

### K-means Clustering

In [ ]:
# simple k means clustering
from sklearn import cluster

kmeans = cluster.KMeans(5)
clustered = kmeans.fit_predict(pixel_matrix)

dims = np.shape(first)
clustered_img = np.reshape(clustered, (dims[0], dims[1]))
plt.imshow(clustered_img)

In [ ]:
ind0, ind1, ind2, ind3 = [np.where(clustered == x)[0] for x in [0, 1, 2, 3]]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

plot_vals = [('r', 'o', ind0),
             ('b', '^', ind1),
             ('g', '8', ind2),
             ('m', '*', ind3)]

for c, m, ind in plot_vals:
    xs = pixel_matrix[ind, 0]
    ys = pixel_matrix[ind, 1]
    zs = pixel_matrix[ind, 2]
    ax.scatter(xs, ys, zs, c=c, marker=m)

ax.set_xlabel('Blue channel')
ax.set_ylabel('green channel')
ax.set_zlabel('Red channel')

In [ ]:
# quick look at color value histograms for pixel matrix from first image
import seaborn as sns
sns.distplot(pixel_matrix[:,0], bins=12)
sns.distplot(pixel_matrix[:,1], bins=12)
sns.distplot(pixel_matrix[:,2], bins=12)

### Matching Features

In [ ]:
img79_1, img79_2, img79_3, img79_4, img79_5 = [plt.imread(train_images_path[n]) for n in range(78, 83)]
img_list = (img79_1, img79_2, img79_3, img79_4, img79_5)

plt.figure(figsize=(8,10))
plt.imshow(img_list[0])
plt.show()

Tracking dimensions across image transforms is annoying, so we'll make a class to do that. Also I'm going to use this brightness normalization transform and visualize the image that way, good test scenario for class.

In [ ]:
class MSImage():
    """Lightweight wrapper for handling image to matrix transforms. No setters,
    main point of class is to remember image dimensions despite transforms."""
    
    def __init__(self, img):
        """Assume color channel interleave that holds true for this set."""
        self.img = img
        self.dims = np.shape(img)
        self.mat = np.reshape(img, (self.dims[0] * self.dims[1], self.dims[2]))

    @property
    def matrix(self):
        return self.mat
        
    @property
    def image(self):
        return self.img
    
    def to_flat_img(self, derived):
        """"Use dims property to reshape a derived matrix back into image form when
        derived image would only have one band."""
        return np.reshape(derived, (self.dims[0], self.dims[1]))
    
    def to_matched_img(self, derived):
        """"Use dims property to reshape a derived matrix back into image form."""
        return np.reshape(derived, (self.dims[0], self.dims[1], self.dims[2]))

In [ ]:
msi79_1 = MSImage(img79_1)
print(np.shape(msi79_1.matrix))
print(np.shape(msi79_1.img))

In [ ]:
plt.imshow(img79_1)

### Brightness Normalization
Brightness Normalization is preprocessing strategy you can apply prior to using strategies to identify materials in a scene, if you want your matching algorithm to be robust across variations in illumination. See [Wu's paper](https://pantherfile.uwm.edu/cswu/www/my%20publications/2004_RSE.pdf).

In [ ]:
def bnormalize(mat):
    """much faster brightness normalization, since it's all vectorized"""
    bnorm = np.zeros_like(mat, dtype=np.float32)
    maxes = np.max(mat, axis=1)
    bnorm = mat / np.vstack((maxes, maxes, maxes)).T
    return bnorm
bnorm = bnormalize(msi79_1.matrix)
bnorm_img = msi79_1.to_matched_img(bnorm)
plt.figure(figsize=(8,10))
plt.imshow(bnorm_img)
plt.show()

In [ ]:
msi79_2 = MSImage(img79_2)
bnorm79_2 = bnormalize(msi79_2.matrix)
bnorm79_2_img = msi79_2.to_matched_img(bnorm79_2)
plt.figure(figsize=(8,10))
plt.imshow(bnorm79_2_img)
plt.show()

In [ ]:
sns.distplot(msi79_1.matrix[:,0], bins=12)
sns.distplot(msi79_1.matrix[:,1], bins=12)
sns.distplot(msi79_1.matrix[:,2], bins=12)

### Using Thresholds with Brightness Normalization
The brightness normalization step is helpful because thresholds that aren't anchored by a preprocessing step end up being arbitrary and can't generalize between scenes even in the same image set, whereas thresholds following brightness normalization tend to pull out materils that stand out from the background more reliably. See the following demonstration:

In [ ]:
plt.figure(figsize=(10,15))
plt.subplot(121)
plt.imshow(img79_1[:,:,0] > 230)
plt.subplot(122)
plt.imshow(img79_1)
plt.show()

In [ ]:
plt.figure(figsize=(10,15))
plt.subplot(121)
plt.imshow(img79_2[:,:,0] > 230)
plt.subplot(122)
plt.imshow(img79_2)
plt.show()

In [ ]:
plt.figure(figsize=(10,15))
plt.subplot(121)
plt.imshow(bnorm79_2_img[:,:,0] > 0.98)
plt.subplot(122)
plt.imshow(img79_2)
plt.show()

In [ ]:
plt.figure(figsize=(10,15))
plt.subplot(121)
plt.imshow(bnorm_img[:,:,0] > 0.98)
plt.subplot(122)
plt.imshow(img79_1)
plt.show()

In [ ]:
plt.subplot(121)
plt.imshow((bnorm79_2_img[:,:,0] > 0.9999) & \
           (bnorm79_2_img[:,:,1] < 0.9999) & \
           (bnorm79_2_img[:,:,2] < 0.9999))
plt.subplot(122)
plt.imshow(img79_2)
plt.show()


In [ ]:

plt.figure(figsize=(10,15))
plt.subplot(121)
plt.imshow(bnorm_img[:,:,0] > 0.995)
plt.subplot(122)
plt.imshow(img79_1)
plt.show()

### Eigen Images

In [ ]:
# from tensorflow.keras.preprocessing import image

# #Add Progressbar
# !pip install progressbar

# from progressbar import ProgressBar
# pbar = ProgressBar()

# # making n X m matrix
# def img2np(list_of_filename, size = (64, 64)):
#     # iterating through each file
#     for fp in pbar(list_of_filename):
# #         fp = path + fn
#         current_image = image.load_img(fp, target_size = size, )
# #                                        color_mode = 'grayscale')
#         # covert image to a matrix
#         img_ts = image.img_to_array(current_image)
#         # turn that into a vector / 1D array
#         img_ts = [img_ts.ravel()]
#         try:
#             # concatenate different images
#             full_mat = np.concatenate((full_mat, img_ts))
#         except UnboundLocalError: 
#             # if not assigned yet, assign one
#             full_mat = img_ts
#     return full_mat

# # run it on our folder
# train_images = img2np(train_images_path)
# train_images

In [ ]:
# from sklearn.decomposition import PCA
# from math import ceil

# def eigenimages(full_mat, title, n_comp = 0.7, size = (64, 64)):
#     # fit PCA to describe n_comp * variability in the class
#     pca = PCA(n_components = n_comp, whiten = True)
#     pca.fit(full_mat)
#     print('Number of PC: ', pca.n_components_)
#     return pca
  
# def plot_pca(pca, size = (64, 64)):
#     # plot eigenimages in a grid
#     n = pca.n_components_
#     fig = plt.figure(figsize=(8, 8))
#     r = int(n**.5)
#     c = ceil(n/ r)
#     for i in range(n):
#         ax = fig.add_subplot(r, c, i + 1, xticks = [], yticks = [])
#         ax.imshow(pca.components_[i].reshape(size), 
#                   cmap='Greys_r')
#     plt.axis('off')
#     plt.show()
    
# plot_pca(eigenimages(train_images, 'Eigen Images'))
# # plot_pca(eigenimages(pnemonia_images, 'PNEUMONIA'))

<div class="list-group" id="list-tab" role="tablist">
<h2 id="advanced-image-exploration" class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Advanced Image Exploration</center></h2>

### Rudimentary Transforms, Edge Detection, Texture

In [ ]:
set144 = [plt.imread(train_images_path[n]) for n in (10000, 11000)]
plt.imshow(set144[0])

In [ ]:
import skimage
from skimage.feature import greycomatrix, greycoprops
from skimage.filters import sobel

### Sobel Edge Detection

A Sobel filter is one means of getting a basic edge magnitude/gradient image. Can be useful to threshold and find prominent linear features, etc. Several other similar filters in skimage.filters are also good edge detectors: `roberts`, `scharr`, etc. and you can control direction, i.e. use an anisotropic version.

In [ ]:
# a sobel filter is a basic way to get an edge magnitude/gradient image
fig = plt.figure(figsize=(8, 8))
plt.imshow(sobel(set144[0][:750,:750,2]))

In [ ]:
from skimage.filters import sobel_h

# can also apply sobel only across one direction.
fig = plt.figure(figsize=(8, 8))
plt.imshow(sobel_h(set144[0][:750,:750,2]), cmap='BuGn')

### GLCM Textures
Processing time can be pretty brutal so we subset the image. We'll create texture images so we can characterize each pixel by the texture of its neighborhood.

GLCM is inherently anisotropic but can be averaged so as to be rotation invariant. For more on GLCM, see [the tutorial](http://www.fp.ucalgary.ca/mhallbey/tutorial.htm).

A good article on use in remote sensing is [here](http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=4660321&url=http%3A%2F%2Fieeexplore.ieee.org%2Fxpls%2Fabs_all.jsp%3Farnumber%3D4660321):

Pesaresi, M., Gerhardinger, A., & Kayitakire, F. (2008). A robust built-up area presence index by anisotropic rotation-invariant textural measure. Selected Topics in Applied Earth Observations and Remote Sensing, IEEE Journal of, 1(3), 180-192.

In [ ]:
sub = set144[0][:150,:150,2]

def glcm_image(img, measure="dissimilarity"):
    """TODO: allow different window sizes by parameterizing 3, 4. Also should
    parameterize direction vector [1] [0]"""
    texture = np.zeros_like(sub)

    # quadratic looping in python w/o vectorized routine, yuck!
    for i in range(img.shape[0] ):  
        for j in range(sub.shape[1] ):  
          
            # don't calculate at edges
            if (i < 3) or \
               (i > (img.shape[0])) or \
               (j < 3) or \
               (j > (img.shape[0] - 4)):          
                continue  
        
            # calculate glcm matrix for 7 x 7 window, use dissimilarity (can swap in
            # contrast, etc.)
            glcm_window = img[i-3: i+4, j-3 : j+4]  
            glcm = greycomatrix(glcm_window, [1], [0],  symmetric = True, normed = True )   
            texture[i,j] = greycoprops(glcm, measure)  
    return texture

    dissimilarity = glcm_image(sub, "dissimilarity")

fig = plt.figure(figsize=(8, 8))
plt.subplot(1,2,1)
plt.imshow(dissimilarity, cmap="bone")
plt.subplot(1,2,2)
plt.imshow(sub, cmap="bone")

### HSV Transform
HSV is useful for identifying shadows and illumination, as well as giving us a means to identify similar objects that are distinct by color between scenes (hue), though there's no guarantee the hue will be stable.

In [ ]:
from skimage import color

hsv = color.rgb2hsv(set144[0])

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.subplot(2,2,1)
plt.imshow(set144[0], cmap="bone")
plt.subplot(2,2,2)
plt.imshow(hsv[:,:,0], cmap="bone")
plt.subplot(2,2,3)
plt.imshow(hsv[:,:,1], cmap='bone')
plt.subplot(2,2,4)
plt.imshow(hsv[:,:,2], cmap='bone')

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.subplot(2,2,1)
plt.imshow(set144[0][:200,:200,:])
plt.subplot(2,2,2)
plt.imshow(hsv[:200,:200,0], cmap="PuBuGn")
plt.subplot(2,2,3)
plt.imshow(hsv[:200,:200,1], cmap='bone')
plt.subplot(2,2,4)
plt.imshow(hsv[:200,:200,2], cmap='bone')

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.imshow(hsv[200:500,200:500,0], cmap='bone')

In [ ]:
hsvmsi = MSImage(hsv)

### Shadow Detection
We can apply a threshold to the V band now to find dark areas that are probably thresholds. Let's look at the distribution of all values then work interactively to find a good filter value.

In [ ]:
sns.distplot(hsvmsi.matrix[:,0], bins=12)
sns.distplot(hsvmsi.matrix[:,1], bins=12)
sns.distplot(hsvmsi.matrix[:,2], bins=12)

In [ ]:
plt.imshow(hsvmsi.image[:,:,2] < 0.4, cmap="plasma")

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.subplot(1,2,1)
plt.imshow(set144[0][:250,:250,:])
plt.subplot(1,2,2)
plt.imshow(hsvmsi.image[:250,:250,2] < 0.4, cmap="plasma")

In [ ]:
fig = plt.figure(figsize=(8, 8))
img2 = plt.imshow(set144[0][:250,:250,:], interpolation='nearest')
img3 = plt.imshow(hsvmsi.image[:250,:250,2] < 0.4, cmap='binary_r', alpha=0.4)
plt.show()

<div class="list-group" id="list-tab" role="tablist">
<h2 id="references" class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>References</center></h2>

1. [Shopee - Data understanding and analysis](https://www.kaggle.com/isaienkov/shopee-data-understanding-and-analysis) 
2. [EDA of Shopee for starter](https://www.kaggle.com/chumajin/eda-of-shopee-for-starter)  
3. [Shopee Product Matching: EDA+Baseline](https://www.kaggle.com/ruchi798/shopee-product-matching-eda-baseline/data)   
4. [Shopee EDA + Understanding Competition!](https://www.kaggle.com/heyytanay/shopee-eda-understanding-competition)    
5. [Exploratory Image Analysis](https://www.kaggle.com/bkamphaus/exploratory-image-analysis#Brightness-Normalization)

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:blue; border:0; color:white' role="tab" aria-controls="home"><center>More Awesome Plots Coming Soon!</center></h2>